## Desafio Engenheiro de Dados Jr - Cidacs

Imports

In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import expr, col, trim, regexp_replace, row_number, lit, when
import pandas as pd
import random

Configuração do SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("DesafioCidacs") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "2") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.sql.shuffle.partitions", "50") \
    .getOrCreate()

# Para otimização de leitura e escrita de dados
spark.conf.set("spark.sql.parquet.compression.codec", "snappy")

# Para melhorar a performance do Spark SQL
spark.conf.set("spark.sql.broadcastTimeout", "1000")

#### Extração do DataBase CNEFE - BAHIA

In [3]:
#Base bahia, código 29
df_principal = spark.read.text("./BASE_CNEFE_BA/base.txt")

In [4]:
df_principal.show(10)

+--------------------+
|               value|
+--------------------+
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
|2900108050000011R...|
+--------------------+
only showing top 10 rows



- Filtrando pela cidade de Salvador

In [4]:
#Sabemos que o código de salvador é 27408, vamos filtrar só o que nos interessa
df_salvador = df_principal.filter(expr("substr(value,3,5)") == '27408')

In [6]:
#Verificando se a filtragem deu certo
df_salvador.show()

In [7]:
#Tamanho da base df_salvador
tam_df_salvador = df_salvador.count()
print(tam_df_salvador)

1097829


#### Formatando DataSet

In [5]:
'''
Vamos filtrar se baseando no Layout do IBGE, baixado no mesmo local da base.
substr(coluna,posição inicial, tamanho)
'''
df_salvador = df_salvador.select( expr("substr(value,17,20)").alias("TP_Logradouro"),
                                    expr("substr(value,37,30)").alias("Titulo_Logradouro"),
                                    expr("substr(value,67,60)").alias("Nome_Logradouro"),
                                    expr("substr(value,127,8)").alias("Num_Logradouro"),
                                    expr("substr(value,12,4)").alias("Setor_Censitario"),
                                    expr("substr(value,142,180)").alias("Complemento")
                                   )

In [ ]:
df_salvador.show()

+--------------------+--------------------+--------------------+--------------+----------------+--------------------+
|       TP_Logradouro|   Titulo_Logradouro|     Nome_Logradouro|Num_Logradouro|Setor_Censitario|         Complemento|
+--------------------+--------------------+--------------------+--------------+----------------+--------------------+
|TRAVESSA            |                 ...|BASILIO MAGALHAES...|      0       |            0001|                 ...|
|TRAVESSA            |                 ...|BASILIO MAGALHAES...|      0       |            0001|                 ...|
|TRAVESSA            |                 ...|BASILIO MAGALHAES...|      0       |            0001|                 ...|
|TRAVESSA            |                 ...|BASILIO MAGALHAES...|      0       |            0001|                 ...|
|TRAVESSA            |                 ...|BASILIO MAGALHAES...|      19      |            0001|                 ...|
|TRAVESSA            |                 ...|BASILIO MAGAL

In [ ]:
#Informações do DataFrame
df_salvador.printSchema()

root
 |-- TP_Logradouro: string (nullable = true)
 |-- Titulo_Logradouro: string (nullable = true)
 |-- Nome_Logradouro: string (nullable = true)
 |-- Num_Logradouro: string (nullable = true)
 |-- Setor_Censitario: string (nullable = true)
 |-- Complemento: string (nullable = true)



#### Criando Base A

In [8]:
"""
Agora vamos tirar uma amostra no tamanho de 100 mil da base de salvador
Vamos fazer uma amostragem aleatória sem reposição usando o método "sample"
"""
tam_amostra_A  = 100000

fracao_amostra = tam_amostra_A/tam_df_salvador # = 0.0910888672097385

#arrendondar "fraction" pra cima e limitar por 100 mil, além de incluir a seed pra manter a Reprodutibilidade

df_A = df_salvador.sample(withReplacement = False, fraction = 0.092, seed = 42).limit(tam_amostra_A) 

In [ ]:
#contagem base A
df_A.count()

100000

- Vamos formatar a base A, deixar ela mais apresentável

In [9]:
#Vamos tirar os espações em excesso dos campos nas laterais da string usando a função trim()

df_A = df_A.withColumn("TP_Logradouro", trim(col("TP_Logradouro")))\
           .withColumn("Titulo_Logradouro", trim(col("Titulo_Logradouro")))\
           .withColumn("Nome_Logradouro", trim(col("Nome_Logradouro")))\
           .withColumn("Num_Logradouro", trim(col("Num_Logradouro")))\
           .withColumn("Setor_Censitario", trim(col("Setor_Censitario")))\
           .withColumn("Complemento", trim(col("Complemento")))

In [ ]:
df_A.show(10)

+-------------+-----------------+-----------------+--------------+----------------+--------------------+
|TP_Logradouro|Titulo_Logradouro|  Nome_Logradouro|Num_Logradouro|Setor_Censitario|         Complemento|
+-------------+-----------------+-----------------+--------------+----------------+--------------------+
|     TRAVESSA|                 |BASILIO MAGALHAES|             0|            0001|                    |
|        LARGO|                 |     DA MARIQUITA|            10|            0001|APARTAMENTO      ...|
|          RUA|                 |          DO MEIO|             0|            0001|                    |
|          RUA|                 |          DO MEIO|             0|            0001|                    |
|          RUA|                 |          DO MEIO|            74|            0001|             SOBRADO|
|          RUA|                 |          DO MEIO|            74|            0001|              TERREO|
|          RUA|                 |          DO MEIO|    

In [10]:
#Agora vamos tirar o espaço em excesso entre as palavra no campo COMPLEMENTO

df_A = df_A.withColumn("Complemento", regexp_replace("Complemento", '\\s+', ' '))

In [11]:
#Também vamos acrescentar id's a essa base usando row_number

windowSpec = Window.orderBy(lit(1))
df_A = df_A.withColumn("Id", row_number().over(windowSpec).cast("int") - 1)

In [12]:
#Vamos colocar o Id no início pois naturalmente ele é a última coluna

df_A = df_A.select("Id",
                    "TP_Logradouro",
                    "Titulo_Logradouro",
                    "Nome_Logradouro",
                    "Num_Logradouro",
                    "Setor_Censitario",
                    "Complemento"
                    )

In [ ]:
#Podemos verificar que os espaços foram retirados e o ID incluído

df_A.show(10)

+---+-------------+-----------------+-----------------+--------------+----------------+---------------+
| Id|TP_Logradouro|Titulo_Logradouro|  Nome_Logradouro|Num_Logradouro|Setor_Censitario|    Complemento|
+---+-------------+-----------------+-----------------+--------------+----------------+---------------+
|  0|     TRAVESSA|                 |BASILIO MAGALHAES|             0|            0001|               |
|  1|        LARGO|                 |     DA MARIQUITA|            10|            0001|APARTAMENTO 202|
|  2|          RUA|                 |          DO MEIO|             0|            0001|               |
|  3|          RUA|                 |          DO MEIO|             0|            0001|               |
|  4|          RUA|                 |          DO MEIO|            74|            0001|        SOBRADO|
|  5|          RUA|                 |          DO MEIO|            74|            0001|         TERREO|
|  6|          RUA|                 |          DO MEIO|         

In [13]:
df_A_pandas = df_A.toPandas()
df_A_pandas.to_csv("baseA.csv", na_rep = "NULL", index = False, encoding = 'utf-8')

Py4JJavaError: An error occurred while calling o104.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 55) (DESKTOP-6IQNCVJ executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2844)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2780)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2779)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2779)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSet

#### Descritiva Simples

 - A base "df_salvador" importada tem um pouco mais de 1M de linhas
 
 - A amostra A tem **100 mil linhas** e conseguimos notar que, todas as colunas são, por enquanto, do tipo **string**

 - Havia excesso de espaços no campo **COMPLEMENTO**,  **LOGRADOURO** e **TITULO**

 - Além disso, há valores ausentes(string vazia) em **COMPLEMENTO** e **TITULO**

In [ ]:
df_A.select(col("Setor_Censitario")).distinct().count()

593

- Observamos também que há **593 setores censitários** na amostra **A**

- Outra observação é que vamos manter as **strings** vazias da amostra **A**, em seguida faremos o mesmo pra **B**, pois as colunas **COMPLEMENTO** e **TITULO** são opcionais e não necessariamente tem valores, portanto usaremos o algoritmo nessas condições.

#### Criando Base B

- Amostragem

In [ ]:
'''
Vamos criar a base B a partir da A, já formatada e menor, isso vai nos trazer mais desempenho
e mais simplicidade para a aplicação do algoritmo
'''

#vamos usar 6 mil registros para a base B

tam_amostra_B = 6000

#fração em relação ao tamanho base A

fracao_amostra_B = tam_amostra_B/tam_amostra_A # 6000/100000 = 0.06

#Amostragem aleatória, arrendodando pra cima e limitando pelo tamanho da amostra

df_B = df_A.sample(fraction = 0.065, seed = 32, withReplacement = False).limit(tam_amostra_B)

In [ ]:
#Contagem

df_B.count()

6000

In [ ]:
#Vamos reformatar o Id, que veio aleatoriamente de acordo com a amostragem

df_B = df_B.withColumn('Id', row_number().over(windowSpec).cast('int') - 1)

In [ ]:
#df resultante

df_B.show(10)

+---+-------------+-----------------+---------------+--------------+----------------+--------------------+
| Id|TP_Logradouro|Titulo_Logradouro|Nome_Logradouro|Num_Logradouro|Setor_Censitario|         Complemento|
+---+-------------+-----------------+---------------+--------------+----------------+--------------------+
|  0|          RUA|                 |   FONTE DO BOI|           176|            0001|APARTAMENTO 303 B...|
|  1|      AVENIDA|                 |   OSWALDO CRUZ|           222|            0001|              LOJA 4|
|  2|          RUA|                 |   FONTE DO BOI|            55|            0002|APARTAMENTO 05 AN...|
|  3|          RUA|                 | BARRO VERMELHO|             0|            0003|APARTAMENTO 204 B...|
|  4|          RUA|                 | BARRO VERMELHO|            93|            0003|     APARTAMENTO 105|
|  5|        PRACA|                 | AUGUSTO SEVERO|             6|            0004|                    |
|  6|          RUA|                 |

#### Inserindo Mil Registros Com Valores Ausentes Em Uma Das Variáveis


- Para fazermos isso vamos usar a seguinte abordagem:
    
    - Vamos criar uma **lista** de valores random baseados **tamanho** da amostra **B**.

    - Inserir o valor None(vazio em python), em uma das colunas, no caso vamos usar a coluna **NUMERO** (Num_Logradouro)

    - Usaremos **.withColumn** com **.when().otherwise()** para comparar os valores da **lista** com o **Id** do **DataFrame** e em seguida inserir os valores ausentes.
  

In [ ]:
#variável com a quantidade de valores ausentes
qtd_ausentes = 1000

#seed para preservar a reprodutibilidade dos dados
random.seed(32)

#Criando lista de valores random
lista_random = random.sample(range(tam_amostra_B), qtd_ausentes)

In [ ]:
#Tamanho lista
print(len(lista_random))

#verificar se os valores são distintos entre si
print(len(set(lista_random)))

1000
1000


In [ ]:
#visualizando
print(lista_random)

[634, 1749, 1184, 2483, 5720, 1948, 4065, 199, 5900, 315, 821, 2660, 4169, 2706, 467, 4282, 5657, 3840, 3038, 4925, 6, 4443, 1035, 4298, 77, 3976, 5985, 1662, 2357, 1652, 3655, 667, 2737, 989, 288, 4304, 916, 5961, 1306, 3595, 164, 3533, 404, 313, 4551, 2892, 1857, 1337, 3833, 2536, 5967, 4033, 1372, 4379, 2451, 4657, 4718, 4716, 674, 271, 4809, 2377, 1878, 2362, 2699, 644, 1998, 5430, 3282, 2370, 3495, 5706, 570, 3905, 74, 1475, 3649, 1334, 5753, 744, 511, 1083, 5421, 2129, 395, 4521, 2474, 1192, 3150, 458, 1585, 4828, 297, 4011, 55, 4131, 4296, 5641, 2681, 2953, 4028, 3921, 2045, 5062, 2644, 4302, 4678, 4167, 1910, 1846, 3141, 3081, 4756, 5922, 3037, 1599, 4176, 1053, 4949, 5606, 2874, 5293, 3035, 1128, 5088, 2029, 8, 4089, 224, 882, 9, 4857, 1367, 2587, 3414, 1854, 3738, 682, 2608, 3555, 1362, 2994, 2481, 676, 5842, 4114, 1726, 1063, 5667, 4787, 2146, 3953, 5573, 3002, 3090, 762, 284, 4977, 1495, 30, 4898, 5909, 1623, 5528, 4341, 2636, 3091, 1807, 937, 3278, 2606, 1560, 5366, 1278, 

In [ ]:
# Verificar se há string vazia na coluna NUMERO(Num_Logradouro)
df_B.filter(df_B['Num_Logradouro'] == "").count()

0

In [ ]:
# Verificar se há valores ausentes na coluna NUMERO(Num_Logradouro)
df_B.filter(col("Num_Logradouro").isNull()).count()

0

In [ ]:
#Inserindo valores ausentes na coluna NUMERO(Num_Logradouro)
df_B = df_B.withColumn("Num_Logradouro", when(col('Id').isin(lista_random), None).otherwise(col("Num_Logradouro")))

In [ ]:
# Verificar se os valores ausentes foram inseridos na coluna NUMERO(Num_Logradouro)
df_B.filter(col("Num_Logradouro").isNull()).count()

1000

In [ ]:
#Visualizando
df_B.show(10)

+---+-------------+-----------------+---------------+--------------+----------------+--------------------+
| Id|TP_Logradouro|Titulo_Logradouro|Nome_Logradouro|Num_Logradouro|Setor_Censitario|         Complemento|
+---+-------------+-----------------+---------------+--------------+----------------+--------------------+
|  0|          RUA|                 |   FONTE DO BOI|           176|            0001|APARTAMENTO 303 B...|
|  1|      AVENIDA|                 |   OSWALDO CRUZ|           222|            0001|              LOJA 4|
|  2|          RUA|                 |   FONTE DO BOI|            55|            0002|APARTAMENTO 05 AN...|
|  3|          RUA|                 | BARRO VERMELHO|             0|            0003|APARTAMENTO 204 B...|
|  4|          RUA|                 | BARRO VERMELHO|            93|            0003|     APARTAMENTO 105|
|  5|        PRACA|                 | AUGUSTO SEVERO|             6|            0004|                    |
|  6|          RUA|                 |

#### Supressão De Palavras (aleatória) Nas Variáveis

- Para fazermos isso vamos usar a seguinte abordagem:

    - Primeiro vamos transforamr o **DF Spark** em um **DF pandas**, pra melhorar o fluxo de trabalho e pra facilitar as manipulações.

    - Vamos criar as **funções** que vão retirar **uma** e **duas palavras aleátorias** nas variáveis **Nome_Logradouro** e **Complemento**.

    - Vamos criar **duas listas**, com **valores aleatórios**, baseadas em índices do tamanho da base B. Isso vai permitir modificar mais facilmente os valores de determinada variável.

    - Além disso, vamos exportar a base, somente com os campos **Nome_Logradouro** e **Complemento**, usando o Pandas e vamos ler essa base usando o Spark. Isso vai evitar conflitos e má formatação na mudança de frameworks.

    - Ao final vamos retirar as antigas colunas da **base B** e vamos inserir as novas usando o **FULL JOIN**

In [ ]:
df_B_pd = df_B.toPandas()

In [ ]:
df_B_pd.head()

,Id,TP_Logradouro,Titulo_Logradouro,Nome_Logradouro,Num_Logradouro,Setor_Censitario,Complemento
0,0,RUA,,FONTE DO BOI,176,0001,APARTAMENTO 303 BLOCO B
1,1,AVENIDA,,OSWALDO CRUZ,222,0001,LOJA 4
2,2,RUA,,FONTE DO BOI,55,0002,APARTAMENTO 05 ANDAR 1
3,3,RUA,,BARRO VERMELHO,0,0003,APARTAMENTO 204 BLOCO B
4,4,RUA,,BARRO VERMELHO,93,0003,APARTAMENTO 105


- Funções Usadas

In [ ]:
#Função para supressão de uma palavra
def supressao_palavra(p):
    random.seed(42)
    if p is not None:
        palavras = p.split()
        if palavras:
            indice = random.randint(0,len(palavras) - 1)
            del palavras[indice]
            return " ".join(palavras)
        else:
            return p
    else:
        return None

    

In [ ]:
#Função para supressão de duas palavras
def supressao_palavra_2(p):
    if p is not None:
        palavras = p.split()
        for i in range(2):
            if palavras:
                indice = random.randint(0,len(palavras) - 1)
                del palavras[indice]
            else:
                return p
        return " ".join(palavras)
    else:
        return None

#### Supressão De **Uma Palavra** em 2 Mil Registros Na Coluna **Nome_Logradouro**

In [ ]:
random.seed(33)

#Criando lista de valores random
indices_random_1 = random.sample(range(tam_amostra_B), 2000)

In [ ]:
print(indices_random_1)

[4672, 1370, 5179, 1910, 2271, 3923, 5340, 4363, 4320, 1527, 5338, 5055, 4123, 2629, 5628, 4327, 3940, 3647, 2311, 5512, 5206, 579, 2468, 3389, 2533, 4245, 4064, 430, 4977, 5547, 3490, 2297, 2617, 4551, 5581, 2012, 960, 458, 2615, 2330, 1747, 542, 2997, 1009, 5353, 2635, 1988, 4200, 2234, 2622, 52, 2898, 4298, 5394, 2493, 4546, 3229, 3187, 1519, 5631, 901, 269, 2986, 5139, 5594, 4584, 1014, 518, 1220, 764, 980, 2593, 24, 471, 3358, 2256, 2852, 1662, 1579, 5544, 5355, 3182, 3750, 4896, 3878, 2933, 4830, 4415, 3721, 2523, 3982, 575, 79, 3941, 3111, 3106, 2961, 909, 5469, 1997, 121, 4442, 5199, 3633, 2583, 5751, 1337, 4054, 5015, 3959, 274, 5659, 716, 4929, 2307, 3790, 2291, 3457, 4152, 3329, 2134, 3870, 5096, 3947, 3177, 168, 3253, 2601, 2662, 1373, 864, 1517, 5830, 2619, 5095, 2344, 5108, 1412, 2896, 3506, 3555, 2742, 5293, 3016, 4805, 5381, 4189, 285, 5120, 105, 1867, 3937, 755, 2214, 4778, 640, 4693, 961, 1627, 1545, 1630, 4988, 585, 528, 5569, 90, 4699, 3561, 4326, 5568, 3042, 55, 12

In [ ]:
#Aplicando a função(supressao_palavra) nos indices contidos na lista
df_B_pd.loc[indices_random_1,'Nome_Logradouro'] = df_B_pd.loc[indices_random_1,'Nome_Logradouro'].apply(supressao_palavra)

#### Supressão De **Duas Palavras** em 3 Mil Registros Na Coluna **Complemento**

In [ ]:
random.seed(31)

#Criando lista, com tamanho 3 mil, de valores random
indices_random_2 = random.sample(range(tam_amostra_B), 3000)

In [ ]:
print(indices_random_2)

[100, 3846, 920, 3218, 1153, 5602, 354, 1136, 921, 4384, 1898, 5828, 1143, 1207, 271, 5424, 496, 1118, 1893, 4388, 3667, 4309, 3379, 1678, 4858, 762, 967, 166, 3300, 2797, 1626, 1686, 2705, 3210, 2989, 4889, 2019, 1723, 5930, 1777, 3410, 4791, 5429, 4484, 491, 422, 1473, 2912, 1103, 1670, 1209, 3083, 248, 705, 4351, 3593, 5334, 1788, 1701, 3542, 3010, 1587, 2708, 5250, 1795, 3949, 339, 2157, 2752, 2403, 2738, 4684, 3245, 3947, 5298, 5331, 2591, 1746, 780, 2509, 1444, 4586, 5109, 4471, 5525, 2417, 766, 1421, 1579, 235, 4926, 3069, 5696, 783, 5485, 3270, 2333, 2503, 5319, 1612, 933, 3725, 1659, 2735, 3642, 5971, 397, 4601, 5849, 966, 2192, 809, 5219, 977, 1590, 1647, 2746, 1329, 5844, 1244, 3896, 474, 4080, 5010, 4194, 2277, 3050, 5019, 4016, 5211, 4838, 311, 917, 3976, 1129, 4966, 5676, 4076, 201, 2086, 4012, 3849, 2856, 1945, 91, 792, 3073, 4773, 2314, 3652, 1442, 369, 1082, 1022, 4646, 5028, 409, 5029, 1691, 1583, 2447, 5646, 1505, 4542, 3726, 4575, 5692, 4205, 4295, 5813, 4124, 3288,

In [ ]:
#Aplicando a função(supressao_palavra_2) nos indices contidos na lista 
df_B_pd.loc[indices_random_2,'Complemento'] = df_B_pd.loc[indices_random_2,'Complemento'].apply(supressao_palavra_2)

 - Visualizando

In [ ]:
df_B_pd.head()

,Id,TP_Logradouro,Titulo_Logradouro,Nome_Logradouro,Num_Logradouro,Setor_Censitario,Complemento
0,0,RUA,,FONTE DO BOI,176,0001,APARTAMENTO 303 BLOCO B
1,1,AVENIDA,,OSWALDO CRUZ,222,0001,
2,2,RUA,,FONTE DO BOI,55,0002,ANDAR 1
3,3,RUA,,BARRO VERMELHO,0,0003,APARTAMENTO 204 BLOCO B
4,4,RUA,,BARRO VERMELHO,93,0003,APARTAMENTO 105


 #### Exportando Base Com Pandas e Lendo Com Spark

In [ ]:
#Exportando as colunas de interesse
df_B_pd[['Nome_Logradouro','Complemento']].to_csv("Log_Comp.csv", header = True, index = False, na_rep= None)

In [ ]:
#Lendo com spark
df_Log_Comp = spark.read.csv('Log_Comp.csv', inferSchema = False, header = True, nullValue= None)

 - Recolocando Strings Vazias Que Foram Perdidas

In [ ]:
#Nas colunas "Nome_Logradouro" e "Complemento" onde tiver nulo substitui por ""
df_Log_Comp = df_Log_Comp.withColumn("Nome_Logradouro" , when(col("Nome_Logradouro").isNull(),"").otherwise(col("Nome_Logradouro")))\
                         .withColumn("Complemento", when(col("Complemento").isNull(),"").otherwise(col("Complemento")))

In [ ]:
#Colocando indices a base de dados
df_Log_Comp = df_Log_Comp.withColumn('Id', row_number().over(windowSpec).cast('int') - 1)

In [ ]:
df_Log_Comp.show(10)

+---------------+--------------------+---+
|Nome_Logradouro|         Complemento| Id|
+---------------+--------------------+---+
|   FONTE DO BOI|APARTAMENTO 303 B...|  0|
|   OSWALDO CRUZ|                    |  1|
|   FONTE DO BOI|             ANDAR 1|  2|
| BARRO VERMELHO|APARTAMENTO 204 B...|  3|
| BARRO VERMELHO|     APARTAMENTO 105|  4|
| AUGUSTO SEVERO|                    |  5|
|        TAMOIOS|               2 201|  6|
|         ARGOLO|                    |  7|
|         MAMUTE|                    |  8|
|        VINHAIS|     APARTAMENTO 201|  9|
+---------------+--------------------+---+
only showing top 10 rows



 #### Fazendo As Alterações Na Base **B**

In [ ]:
#retirando as antigas colunas
df_B = df_B.drop("Nome_Logradouro", "Complemento")

In [ ]:
#Fazendo o join(FULL) entre as duas tabelas baseadas no "Id"
df_B = df_B.join(df_Log_Comp, on = "Id", how = "full")

In [ ]:
#Reorganizando a tabela
df_B = df_B.select("Id",
                    "TP_Logradouro",
                    "Titulo_Logradouro",
                    "Nome_Logradouro",
                    "Num_Logradouro",
                    "Setor_Censitario",
                    "Complemento"
                    )

In [ ]:
#Verificando alterações
df_B.show(10)

+---+-------------+-----------------+---------------+--------------+----------------+--------------------+
| Id|TP_Logradouro|Titulo_Logradouro|Nome_Logradouro|Num_Logradouro|Setor_Censitario|         Complemento|
+---+-------------+-----------------+---------------+--------------+----------------+--------------------+
|  0|          RUA|                 |   FONTE DO BOI|           176|            0001|APARTAMENTO 303 B...|
|  1|      AVENIDA|                 |   OSWALDO CRUZ|           222|            0001|                    |
|  2|          RUA|                 |   FONTE DO BOI|            55|            0002|             ANDAR 1|
|  3|          RUA|                 | BARRO VERMELHO|             0|            0003|APARTAMENTO 204 B...|
|  4|          RUA|                 | BARRO VERMELHO|            93|            0003|     APARTAMENTO 105|
|  5|        PRACA|                 | AUGUSTO SEVERO|             6|            0004|                    |
|  6|          RUA|                 |

In [ ]:
#Exportando base B
df_B_pd = df_B.toPandas()
df_B_pd.to_csv("baseB.csv", na_rep = "NULL", index = False, encoding = 'utf-8')